Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Caliese Beckford"
COLLABORATORS = ""

---

# Basic Folding Algorithm
Keywords: pose_from_sequence(), random move, scoring move, Metropolis, assign(), Pose()

In [1]:
!pip install pyrosettacolabsetup
import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()
import pyrosetta; pyrosetta.init()
from pyrosetta import *
from pyrosetta.teaching import *
init()

Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.19+release.a34b73c40f-cp310-cp310-linux_x86_64.whl


┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENS

## Building the Pose

In this lab, you will be folding a 10 residue protein by building a simple de novo folding algorithm. Start by initializing PyRosetta as usual.

Create a simple poly-alanine `pose` with 10 residues for testing your folding algorithm. Store the pose in a variable called "polyA."

Figure out how to set the pdb information of the "polyA" protein to have name "polyA"

In [3]:
# YOUR CODE HERE
polyA = pose_from_sequence("AAAAAAAAAA")

# YOUR CODE HERE
#raise NotImplementedError()

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.07017 seconds.


ensure that the sequence of the polyA sequence is what you expect

In [4]:
# YOUR CODE HERE
polyA.sequence()

'AAAAAAAAAA'

__Question:__
Check the backbone dihedrals of a few residues (except the first and last) using the `.phi()` and `.psi()` methods in `Pose`. What are the values of $\phi$ and $\psi$ dihedrals? You should see ideal bond lengths and angles, but the dihedrals may not be as realistic. Also try to figure out why we can't get the backbone dihedrals of the first and last residues?

In [17]:
# YOUR CODE HERE
for i in range(2, 10):
  print(i, polyA.phi(i), i, polyA.psi(i))


2 180.0 2 180.0
3 180.0 3 180.0
4 180.0 4 180.0
5 180.0 5 180.0
6 180.0 6 180.0
7 180.0 7 180.0
8 180.0 8 180.0
9 180.0 9 180.0


Use PyMOL to view the `polyA` `Pose`. You should see a long thread-like structure in PyMOL.

In [19]:
# visualize the pose in PyMOL and save the screenshot to submit
polyA.dump_pdb("google_drive/MyDrive/CodeSchool2024/inputs/polyA.pdb")

True

## Building A Basic *de Novo* Folding Algorithm

Now, write a program that implements a Monte Carlo algorithm to optimize the protein conformation. The algorithm will be very simple: make a random move, score the protein, and accept/reject the move based on the Metropolis criterion.

Our main program will include 100 iterations of making a random trial move, scoring the protein, and accepting/rejecting the move. Therefore, we can break this algorithm down into three smaller subroutines: **random, score, and decision.**

### Step 1: Random Move

For the **random** trial move, write a subroutine to choose one residue at random using `random.randint()` and then randomly perturb either the φ or ψ angles by a random number chosen from a Gaussian distribution. Use the Python built-in function `random.gauss()` from the `random` library with a mean of the current angle and a standard deviation of 25°.

In [49]:
import math
import random

def randTrial(polyA):
  rand1 =random.randint(1,10)
  #b = random.randint(10)
  print(rand1)
  #print(rand1)

#current phi and current psi as variables in the gauss thing
  current_phi = polyA.phi(rand1)
  current_psi = polyA.psi(rand1)
  new_phi =  random.gauss(current_phi, 25)
  new_psi = random.gauss(current_psi, 25)
  #print(current_phi)
  #print(current_psi)

  polyA.set_phi(rand1, new_phi)
  polyA.set_psi(rand1, new_psi)

  return polyA
  #print(new_phi)
  #print(new_psi)

randTrial(polyA)

polyA.dump_pdb("google_drive/MyDrive/CodeSchool2024/inputs/perturbed_polyA.pdb")



10


True

### Step 2: Scoring Move

For the **scoring** step, we need to create a scoring function and make a subroutine that simply returns the numerical energy score of the pose.

you may need to look back at the previous lab to recall how to get the score function

You should try to use the standard full-atom (fa) ref2015 score function

In [55]:
# DEFINE A SCORING FUNCTION, YOUR CODE HERE
from pyrosetta.teaching import *


def score(polyA):
  sfxn = get_score_function(True)

  print(sfxn(polyA))
  return sfxn(polyA)

score(polyA)

core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.scoring.etable: Starting energy table calculation
core.scoring.etable: smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: Finished calculating energy tables.
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv
basic.io.database: Database file opened: scoring/score_functions/rama/fd/

36.017467579521565

### Step 3: Accepting/Rejecting Move
For the **decision** step, we need to make a subroutine that either accepts or rejects the new conformatuon based on the Metropolis criterion. The Metropolis criterion has a probability of accepting a move as $P = \exp( -\Delta G / kT )$. When $ΔE ≥ 0$ (in this case change in rosetta energy), the Metropolis criterion probability of accepting the move is $P = \exp( -\Delta G / kT )$. When $ΔE < 0$, the Metropolis criterion probability of accepting the move is $P = 1$. Use $kT = 1$ Rosetta Energy Unit (REU).

In [67]:
import math

def decision(before_pose, after_pose):
  deltaG = score(after_pose) - score(before_pose)
  if (deltaG < 0):
    return after_pose
  else:
    P = math.exp(-deltaG / 1)
    sample_prob = random.uniform(0,1)
    if (sample_prob < P):
        return after_pose
    else:
        return before_pose

  #print(decision(polyA,polyA))

  #polyA.dump_pdb("google_drive/MyDrive/CodeSchool2024/inputs/decision_polyA.pdb")

### Step 4: Execution
Now we can put these three subroutines together in our main program! Write a loop in the main program so that it performs 100 iterations of: making a random trial move, scoring the protein, and accepting/rejecting the move.

After each iteration of the search, output the current pose energy and the lowest energy ever observed. **The final output of this program should be the lowest energy conformation that is achieved at *any* point during the simulation.** Be sure to use `low_pose.assign(pose)` rather than `low_pose = pose`, since the latter will only copy a pointer to the original pose.

In [71]:
import math
import random
from pyrosetta.teaching import *

def basic_folding(polyA):
 # x = 1
  for i in range(100):
   def randTrial(polyA):
    rand1 = random.randint(1,10)
    #b = random.randint(10)
    print(rand1)
    #print(rand1)

#current phi and current psi as variables in the gauss thing
    current_phi = polyA.phi(rand1)
    current_psi = polyA.psi(rand1)
    new_phi =  random.gauss(current_phi, 25)
    new_psi = random.gauss(current_psi, 25)
    #print(current_phi)
    #print(current_psi)

    polyA.set_phi(rand1, new_phi)
    polyA.set_psi(rand1, new_psi)

    return polyA
  #print(new_phi)
  #print(new_psi)

randTrial(polyA)

#polyA.dump_pdb("google_drive/MyDrive/CodeSchool2024/inputs/perturbed_polyA.pdb")

# DEFINE A SCORING FUNCTION, YOUR CODE HERE

def score(polyA):
  sfxn = get_score_function(True)

  print(sfxn(polyA))
  return sfxn(polyA)

score(polyA)

def decision(before_pose, after_pose):
  deltaG = score(after_pose) - score(before_pose)
  if (deltaG < 0):
    return after_pose
  else:
    P = math.exp(-deltaG / 1)
    sample_prob = random.uniform(0,1)
    if (sample_prob < P):
        return after_pose
    else:
        return before_pose

#x = x + 1



5
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
37.11710160541038


Finally, output the last pose and the lowest-scoring pose observed and view them in PyMOL (you'll need to dump the pose into a new pdb for this). Submit a screenshot of this.

Plot the energy and lowest-energy observed vs. cycle number. What are the energies of the initial, last, and lowest-scoring pose? Is your program working? Has it converged to a good solution?


In [ ]:
raise NotImplementedError()


### Exercise 1: Comparing to Alpha Helices
Using the program you wrote previously, force the $A_{10}$ sequence into an ideal α-helix.

**Questions:** Does this helical structure have a lower score than that produced by your folding algorithm above? What does this mean about your sampling or discrimination?

In [ ]:
# Answer the above questions here:

# what are the ideal angles for an alpha-helix? force the protein to be an ideal alpha helix. compare it your lowest pose protein.

### Exercise 2: Optimizing Algorithm
Since your program is a stochastic search algorithm, it may not produce an ideal structure consistently, so try running the simulation multiple times or with a different number of cycles (if necessary). Using a kT of 1, your program may need to make up to 500,000 iterations.

In [ ]:
# Report the lowest energy pose you found and the energy of that pose.
# Dump this pose to a PDB file and visualize it in PyMOL. Submit a screenshot of the pose in PyMOL.

raise NotImplementedError()